In [310]:
# packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import datetime as dt
import re 
import yfinance as yf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import requests
from get_all_tickers import get_tickers as gt
import bs4 as bs  # beautiful soups for web-scraping
import os
import pandas_datareader.data as pdr

In [311]:
# choose stocks to target 
html = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(html.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

tickers = []
for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        ticker = ticker[:-1]
        ticker = ticker.replace('.', '-')
        tickers.append(ticker)

The above tickers are the current tickers in the S&P 500...
- Ideally I would like to include all tickers that have been in the S&P 500 over the past 5 years or so, including those that have delisted.
- How to deal with stock ticker name changes?

In [327]:
# fundamental data of companies in S&P 500 sourced from Bloomberg Terminal
SP_500_data = pd.read_excel('SP500_fundamental_data.xlsx', index_col = 0, header=[0,1])
SP_500_data = SP_500_data.rename(str.strip, axis='columns')  # strip trailing whitespace from column names
SP_500_data

A                                                    \
dates      RETURN_ON_ASSET CF_CASH_FROM_OPER BS_CUR_ASSET_REPORT BS_CUR_LIAB   
2015-01-31          4.0976             -20.0              3654.0       928.0   
2015-04-30          3.4711             183.0              3620.0       930.0   
2015-07-31          3.2216             108.0              3478.0       853.0   
2015-10-31          4.3840             237.0              3686.0       976.0   
2016-01-31          6.1939             111.0              3399.0       947.0   
2016-04-30          6.1520             254.0              3488.0      1133.0   
2016-07-31          6.3530             194.0              3530.0      1151.0   
2016-10-31          6.0499             234.0              3635.0       945.0   
2017-01-31          6.7088             116.0              3635.0      1089.0   
2017-04-30          7.4348             257.0              3800.0      1187.0   
2017-07-31          7.9150             228.0              3996.0      1241.0   
2017-10-31          8.4340             288.0              4169.0      1263.0   
2018-01-31          2.3657             215.0              4397.0      1361.0   
2018-04-30          2.8214             303.0              4525.0      1365.0   
2018-07-31          3.5882             197.0              3667.0      1014.0   
2018-10-31          3.7249             372.0              3848.0      1171.0   
2019-01-31         12.9178             213.0              3712.0      1095.0   
2019-04-30         12.5463             252.0              3812.0      1118.0   
2019-07-31         12.6311             242.0              3457.0      1622.0   
2019-10-31         11.9046             314.0              3189.0      2080.0   
2020-01-31          8.2805             -59.0              3102.0      1892.0   
2020-04-30          7.3930             313.0              3171.0      1945.0   
2020-07-31          7.6055             290.0              3245.0      1314.0   
2020-10-31          7.5371             377.0              3415.0      1467.0   
NaT                    NaN               NaN                 NaN         NaN   

                                                                       AAL  \
dates      TOT_DEBT_TO_TOT_ASSET ASSET_TURNOVER NET_INCOME RETURN_ON_ASSET   
2015-01-31               22.0508         0.4479       63.0          7.3709   
2015-04-30               22.3421         0.4383       87.0          9.0096   
2015-07-31               22.8244         0.4590      111.0         10.6063   
2015-10-31               22.1286         0.4415      140.0         16.6085   
2016-01-31               23.7332         0.5452      121.0         15.2654   
2016-04-30               24.7251         0.5442       91.0         13.3928   
2016-07-31               24.3988         0.5507      124.0         11.3561   
2016-10-31               24.4290         0.5503      126.0          5.3687   
2017-01-31               25.3049         0.5590      168.0          4.5174   
2017-04-30               25.4865         0.5524      164.0          4.2726   
2017-07-31               25.1907         0.5494      175.0          4.1620   
2017-10-31               23.8666         0.5514      177.0          2.4640   
2018-01-31               24.6608         0.5572     -320.0          2.0792   
2018-04-30               24.0779         0.5619      205.0          1.4968   
2018-07-31               21.5475         0.5790      236.0          0.9597   
2018-10-31               21.0631         0.5792      195.0          2.4805   
2019-01-31               20.0849         0.5651      504.0          2.5108   
2019-04-30               19.9291         0.5637      182.0          2.6844   
2019-07-31               20.8464         0.5997      191.0          2.7959   
2019-10-31               25.4655         0.5739      194.0          2.7966   
2020-01-31               27.9339         0.5675      197.0         -1.2399   
2020-04-30               28.3131         0.5668      101.0         -5.

In [357]:
# price per earnings ratio 
SP500_PE_ratio = pd.read_excel('SP500_PE_ratio_monthly_10-20.xlsx', index_col=0, header = [0,1])
SP500_PE_ratio = SP500_PE_ratio.rename(str.strip, axis='columns')  # strip trailing whitespace from column names
SP500_PE_ratio = SP500_PE_ratio.iloc[::3, :]  # quarterly data
SP500_PE_ratio.index = [str(x.year) + "_" + str(x.quarter) for x in SP500_PE_ratio.index]  # clean date index
#SP500_PE_ratio = SP500_PE_ratio.stack(0)

In [349]:
SP500_PE_ratio.index

DatetimeIndex(['2010-01-29', '2010-04-30', '2010-07-30', '2010-10-29',
               '2011-01-31', '2011-04-29', '2011-07-29', '2011-10-31',
               '2012-01-31', '2012-04-30', '2012-07-31', '2012-10-31',
               '2013-01-31', '2013-04-30', '2013-07-31', '2013-10-31',
               '2014-01-31', '2014-04-30', '2014-07-31', '2014-10-31',
               '2015-01-30', '2015-04-30', '2015-07-31', '2015-10-30',
               '2016-01-29', '2016-04-29', '2016-07-29', '2016-10-31',
               '2017-01-31', '2017-04-28', '2017-07-31', '2017-10-31',
               '2018-01-31', '2018-04-30', '2018-07-31', '2018-10-31',
               '2019-01-31', '2019-04-30', '2019-07-31', '2019-10-31',
               '2020-01-31', '2020-04-30', '2020-07-31', '2020-10-30'],
              dtype='datetime64[ns]', freq=None)

In [344]:
SP500_PE_ratio

A                 AAL                 AAP                AAPL  \
Dates  PE_RATIO Last Price PE_RATIO Last Price PE_RATIO Last Price PE_RATIO   
2010_1  31.5430    20.0439      NaN        NaN  13.9863      39.45  18.7617   
2010_2  26.4607    25.9291      NaN        NaN  14.8815      45.10  22.1664   
2010_3  10.2540    19.9724      NaN        NaN  15.8690      53.53  19.3712   
2010_4  10.5863    24.8850      NaN        NaN  17.2532      64.98  19.8594   
2011_1  11.5778    29.9121      NaN        NaN  16.0762      63.94  18.9458   
2011_2  12.5717    35.6900      NaN        NaN  15.7824      65.46  16.5982   
2011_3  11.3841    30.1481      NaN        NaN  12.3923      54.97  15.4584   
2011_4   9.5479    26.5083      NaN        NaN  13.5484      65.07  14.6288   
2012_1  10.3329    30.3697      NaN        NaN  14.8907      76.64  13.0014   
2012_2   9.9062    30.1624      NaN        NaN  16.4620      91.80  14.2463   
2012_3   9.4253    27.3807      NaN        NaN  12.8563      70.15  14.3573   
2012_4  10.4614    25.7360      NaN        NaN  13.4957      70.94  13.4760   
2013_1  13.8895    32.0216      NaN        NaN  14.0656      73.52  10.3245   
2013_2  13.3725    29.6332      NaN        NaN  16.4410      83.88  10.5693   
2013_3  15.1421    31.9858      NaN        NaN  15.4132      82.49  11.3012   
2013_4  14.7675    36.2978      NaN        NaN  17.6417      99.18  13.1807   
2014_1  18.1515    41.5823  15.9203     33.550  20.2454     114.81  12.4195   
2014_2  21.8234    38.6433  11.4375     35.070  19.7350     121.29  14.1192   
2014_3  27.0660    40.1092  11.7352     38.850  18.5015     121.11  15.4695   
2014_4  32.4792    39.5300  12.3528     41.350  21.3731     146.96  16.8500   
2015_1  32.4291    37.7700   8.5971     49.080  21.4865     159.00  15.8381   
2015_2  31.2193    41.3700   6.9499     48.285  18.9655     143.00  15.4890   
2015_3  28.9413    40.9500   5.0286     40.100  22.5078     174.21  14.0140   
2015_4  25.8573    37.7600   5.0936     46.220  25.3621     198.43  12.9746   
2016_1  24.9317    37.6500   4.3477     38.990  20.1983     152.05  10.3801   
2016_2  26.7484    40.9200   4.0182     34.690  20.3675     156.10  10.4735   
2016_3  30.7221    48.1100   4.0666     35.500  23.2525     169.86  12.2348   
2016_4  27.0396    43.5700   5.2483     40.600  19.7817     140.08  13.8189   
2017_1  27.9601    48.9700   6.6423     44.250  23.9619     164.24  14.5238   
2017_2  28.5429    55.0500   6.8397     42.620  23.9378     142.14  16.8132   
2017_3  31.6935    59.7900   9.1023     50.440  20.0089     112.01  16.9198   
2017_4  34.5991    68.0300   8.8837     46.820  15.4658      81.74  18.5609   
2018_1  34.5984    73.4300  10.3787     54.320  22.6928     116.99  17.4063   
2018_2  29.3237    65.7400   8.3866     42.930  20.1778     114.45  16.1119   
2018_3  25.2856    66.0400   8.3357     39.540  23.5032     141.23  17.4577   
2018_4  23.2181    64.7900   7.9161     35.080  24.7186     159.76  18.4960   
2019_1  26.9488    76.0500   7.7958     35.770  23.2600     159.20  13.7041   
2019_2  27.7418    78.5000   7.7472     34.180  23.0870     166.32  16.8973   
2019_3  24.9152    69.4100   6.7485     30.510  20.5477     150.64  18.1405   
2019_4  27.4081    75.7500   6.3160     30.060  21.5216     162.48  20.9836   
2020_1  31.0234    82.5600   5.4508     26.840  16.5628     131.75  24.4544   
2020_2  29.7452    76.6600   6.7895     12.010  18.5914     120.91  23.0305   
2020_3  37.4695    96.3300   7.5301     11.120  20.1773     150.14  32.2996   
2020_4  38.6665   102.0900   7.5301     11.280  18.0581     147.28  33.4013   

                      ABBV             ...      YUM                 ZBH  \
Dates  Last Price PE_RATIO Last Price  ... PE_RATIO Last Price PE_RATIO   
2010_1      6.861      NaN        NaN  ...   9.8074    24.5988  14.2753   
2010_2      9.326      NaN        NaN  ...  11.9963    30.5023  15.0742   
2010_3      9.188      NaN        NaN  ...  12.6121    29.6969  13.8511   
2010

In [352]:
m_rets = pd.DataFrame(np.zeros((44,len(tickers))), index = SP500_PE_ratio.index, columns = tickers)

for i in range(len(tickers)):
    m_rets.iloc[:,i] = SP500_PE_ratio.iloc[:,i] / SP500_PE_ratio.iloc[:,i].shift(1) - 1

m_rets = m_rets.shift(-1)
m_rets.index = [str(x.year) + "_" + str(x.quarter) for x in m_rets.index]  # clean date index
m_rets

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,WLTW,WYNN,XEL,XLNX,XYL,YUM,ZBRA,ZBH,ZION,ZTS
2010_1,-0.161123,0.293616,NaN,NaN,0.064005,0.143219,0.181471,0.359277,NaN,NaN,...,0.100176,-0.110851,0.099064,0.143892,0.125234,-0.035165,0.172287,0.162906,0.191192,0.402184
2010_2,-0.612482,-0.229730,NaN,NaN,0.066358,0.186918,-0.126101,-0.014797,NaN,NaN,...,-0.058849,-0.154988,-0.089262,0.005445,0.045266,-0.241846,-0.148728,-0.101659,-0.150065,-0.201915
2010_3,0.032407,0.245969,NaN,NaN,0.087227,0.213899,0.025202,0.169460,NaN,NaN,...,-0.079424,-0.246890,-0.199227,0.262338,0.259039,0.005750,0.050575,0.050156,0.177073,0.063813
2010_4,0.093659,0.202013,NaN,NaN,-0.068219,-0.016005,-0.046003,0.127873,NaN,NaN,...,0.119321,0.089274,0.228018,0.009902,0.117703,0.187891,0.170460,-0.047168,0.075652,0.365522
2011_1,0.085845,0.193163,NaN,NaN,-0.018275,0.023772,-0.123911,0.026240,NaN,NaN,...,0.305860,0.034618,0.082973,0.105881,0.211462,0.030730,0.091980,-0.101443,0.005255,-0.054489
2011_2,-0.094466,-0.155279,NaN,NaN,-0.214803,-0.160251,-0.068670,0.121332,NaN,NaN,...,-0.006907,0.067848,0.145409,-0.216371,-0.139828,-0.191122,-0.147406,-0.223966,-0.108162,-0.235513
2011_3,-0.161295,-0.120731,NaN,NaN,0.093292,0.183737,-0.053667,0.036570,NaN,NaN,...,-0.022130,-0.002235,0.083157,-0.038892,0.043620,-0.074887,-0.023494,-0.100411,-0.095131,-0.056258
2011_4,0.082217,0.145668,NaN,NaN,0.099074,0.177809,-0.111246,0.127767,NaN,NaN,...,-0.003556,-0.003012,0.060033,-0.076642,-0.015836,0.020504,0.090479,0.092586,0.124564,0.096123
2012_1,-0.041295,-0.006826,NaN,NaN,0.105522,0.197808,0.095751,0.279887,NaN,NaN,...,-0.014601,0.167982,0.257197,0.099957,0.155368,0.062087,0.082029,0.073656,0.100576,-0.013170
2012_2,-0.048545,-0.092224,NaN,NaN,-0.219032,-0.235839,0.007791,0.045385,NaN,NaN,...,0.060587,-0.217030,-0.167237,-0.063122,0.013470,-0.084228,-0.052980,-0.067785,-0.109098,-0.136112


In [359]:
pd.merge(SP500_PE_ratio, 
         m_rets,
         left_index=True,
         right_index=True)


/var/folders/nc/rw9wphy93wld5jj0hn7glwbm0000gn/T/ipykernel_21525/2896477675.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  pd.merge(SP500_PE_ratio,


,"(A, PE_RATIO)","(A, Last Price)","(AAL, PE_RATIO)","(AAL, Last Price)","(AAP, PE_RATIO)","(AAP, Last Price)","(AAPL, PE_RATIO)","(AAPL, Last Price)","(ABBV, PE_RATIO)","(ABBV, Last Price)",...,WLTW,WYNN,XEL,XLNX,XYL,YUM,ZBRA,ZBH,ZION,ZTS
2010_1,31.5430,20.0439,NaN,NaN,13.9863,39.45,18.7617,6.861,NaN,NaN,...,0.100176,-0.110851,0.099064,0.143892,0.125234,-0.035165,0.172287,0.162906,0.191192,0.402184
2010_2,26.4607,25.9291,NaN,NaN,14.8815,45.10,22.1664,9.326,NaN,NaN,...,-0.058849,-0.154988,-0.089262,0.005445,0.045266,-0.241846,-0.148728,-0.101659,-0.150065,-0.201915
2010_3,10.2540,19.9724,NaN,NaN,15.8690,53.53,19.3712,9.188,NaN,NaN,...,-0.079424,-0.246890,-0.199227,0.262338,0.259039,0.005750,0.050575,0.050156,0.177073,0.063813
2010_4,10.5863,24.8850,NaN,NaN,17.2532,64.98,19.8594,10.745,NaN,NaN,...,0.119321,0.089274,0.228018,0.009902,0.117703,0.187891,0.170460,-0.047168,0.075652,0.365522
2011_1,11.5778,29.9121,NaN,NaN,16.0762,63.94,18.9458,12.119,NaN,NaN,...,0.305860,0.034618,0.082973,0.105881,0.211462,0.030730,0.091980,-0.101443,0.005255,-0.054489
2011_2,12.5717,35.6900,NaN,NaN,15.7824,65.46,16.5982,12.437,NaN,NaN,...,-0.006907,0.067848,0.145409,-0.216371,-0.139828,-0.191122,-0.147406,-0.223966,-0.108162,-0.235513
2011_3,11.3841,30.1481,NaN,NaN,12.3923,54.97,15.4584,13.946,NaN,NaN,...,-0.022130,-0.002235,0.083157,-0.038892,0.043620,-0.074887,-0.023494,-0.100411,-0.095131,-0.056258
2011_4,9.5479,26.5083,NaN,NaN,13.5484,65.07,14.6288,14.456,NaN,NaN,...,-0.003556,-0.003012,0.060033,-0.076642,-0.015836,0.020504,0.090479,0.092586,0.124564,0.096123
2012_1,10.3329,30.3697,NaN,NaN,14.8907,76.64,13.0014,16.303,NaN,NaN,...,-0.014601,0.167982,0.257197,0.099957,0.155368,0.062087,0.082029,0.073656,0.100576,-0.013170
2012_2,9.9062,30.1624,NaN,NaN,16.4620,91.80,14.2463,20.866,NaN,NaN,...,0.060587,-0.217030,-0.167237,-0.063122,0.013470,-0.084228,-0.052980,-0.067785,-0.109098,-0.136112


In [253]:
# fiscal quarter return data
ticker_data = yf.download(tickers, start="2015-01-01", end="2020-12-31")  # daily
qtr_ticker_data = yf.download(tickers, start="2015-01-01", end="2020-12-31", interval="3mo")  # quarterly

[*********************100%***********************]  505 of 505 completed

1 Failed download:
- OGN: Data doesn't exist for startDate = 1420088400, endDate = 1609390800
[*********************100%***********************]  505 of 505 completed

1 Failed download:
- OGN: Data doesn't exist for startDate = 1420088400, endDate = 1609390800


In [249]:
data_close = ticker_data.loc[:,'Adj Close'].copy()  # daily close price
qtr_data_close = qtr_ticker_data.loc[:, 'Adj Close'].copy()  # quarterly close price

# daily returns 
N = len(data_close.index)  # total number of days
daily_returns = pd.DataFrame(np.zeros((N,len(tickers))), index = data_close.index, columns = tickers)

for i in range(len(tickers)):
    daily_returns.iloc[:,i] = data_close.iloc[:,i] / data_close.iloc[:,i].shift(1) - 1

In [252]:
qtr_data_close

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,39.02145,50.009235,145.387161,28.247126,42.735256,101.290001,71.580002,40.50449,82.195671,73.940002,...,28.443077,37.304779,61.126945,48.610367,32.007053,49.787022,111.423798,90.720001,23.839111,44.195232
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# gross margin 

In [87]:
# compute Piotroski F-score given S&P 500 ticker
ROA = SP_500_data["A"]["RETURN_ON_ASSET"]
CFO = SP_500_data["A"]["CF_CASH_FROM_OPER"]
NET_INCOME = SP_500_data["A"]["NET_INCOME"]
TOT_CURR_ASSET = SP_500_data["A"]["BS_CUR_ASSET_REPORT"]

dROA = ROA.diff(1)
ACCRUAL = (NET_INCOME - CFO)/TOT_CURR_ASSET.shift(1)


In [88]:
ACCRUAL

2015-01-31         NaN
2015-04-30   -0.026273
2015-07-31    0.000829
2015-10-31   -0.027890
2016-01-31    0.002713
2016-04-30   -0.047955
2016-07-31   -0.020069
2016-10-31   -0.030595
2017-01-31    0.014305
2017-04-30   -0.025585
2017-07-31   -0.013947
2017-10-31   -0.027778
2018-01-31   -0.128328
2018-04-30   -0.022288
2018-07-31    0.008619
2018-10-31   -0.048268
2019-01-31    0.075624
2019-04-30   -0.018858
2019-07-31   -0.013379
2019-10-31   -0.034712
2020-01-31    0.080276
2020-04-30   -0.068343
2020-07-31   -0.028698
2020-10-31   -0.047766
NaT                NaN
dtype: float64

In [215]:
# compute Piotroski F-score

def F_SCORE(ticker, start_date, end_date):

    #Balance Sheet
        url_1 = "https://financialmodelingprep.com/api/v3/financials/"
        balance_sheet = "balance-sheet-statement/"
        url_2 = f"{ticker}?period=quarter&apikey=f0060c7a7b275396c21cb98f4985f3ae"
        BS = requests.get(url_1 + balance_sheet + url_2)
        BS = BS.json()
        N = len(bs['financials'])

        for i in range(N):
            if re.search(year+"-1\d-\d\d", bs["financials"][i]['date']):
                a,b,c = i,i+1,i+2
                break

        print("year of study is "+bs["financials"][a]['date'])

        # target year
        long_term_debt = float(bs["financials"][a]['Long-term debt'])
        total_assets = float(bs["financials"][a]['Total assets'])
        current_assets = float(bs["financials"][a]['Total current assets'])
        current_assets = current_assets if current_assets != 0 else 1
        current_liabilities = float(bs["financials"][a]['Total current liabilities'])
        current_liabilities = current_liabilities if current_liabilities != 0 else 1

        # prior year
        long_term_debt_py = float(bs["financials"][b]['Long-term debt'])
        total_assets_py = float(bs["financials"][b]['Total assets'])
        current_assets_py = float(bs["financials"][b]['Total current assets'])
        current_assets_py = current_assets_py if current_assets_py != 0 else 10
        current_liabilities_py = float(bs["financials"][b]['Total current liabilities'])
        current_liabilities_py = current_liabilities_py if current_liabilities_py != 0 else 1

        # prior year * 2 
        total_assets_py2 = float(bs["financials"][c]['Total assets'])

        # Income statement
        income_statement = "income-statement/"
        IS = requests.get(url_1 + income_statement + url_2)
        IS = IS.json()

        # Year 2018
        revenue = float(IS['financials'][a]['Revenue'])
        gross_profit = float(IS['financials'][a]['Gross Profit'])
        net_income = float(IS['financials'][a]['Net Income'])

        # Previous year (2017)
        revenue_py = float(IS['financials'][b]['Revenue'])
        gross_profit_py = float(IS['financials'][b]['Gross Profit'])
        net_income_py = float(IS['financials'][b]['Net Income'])
        
        # Cashflow
        cashflow = "cash-flow-statement/"
        CF = requests.get(url_1 + cashflow + url_2)
        CF = CF.json()
        cashflow_op = float(CF['financials'][a]["Operating Cash Flow"])

        #Piotroski F-score

        ROA_FS = int(net_income/((total_assets + total_assets_py)/2)>0)
        CFO_FS = int(cashflow_op>0)
        ROA_D_FS = int((net_income/((total_assets + total_assets_py)/2))>(net_income_py/((total_assets_py + total_assets_py2))))
        CFO_ROA_FS = int((cashflow_op/total_assets)>(net_income/((total_assets + total_assets_py)/2)))
        LTD_FS = int(long_term_debt <= long_term_debt_py)
        CR_FS = int((current_assets/current_liabilities)>(current_assets_py/current_liabilities_py))
        DILUTION_FS = int(float(IS['financials'][a]['Weighted Average Shs Out'])<=float(IS['financials'][b]['Weighted Average Shs Out']))
        GM_FS = int(gross_profit/revenue>gross_profit_py/revenue_py)
        ATO_FS = int((revenue/((total_assets + total_assets_py)/2))>(revenue_py/((total_assets_py + total_assets_py2))))
        f_score = ROA_FS + CFO_FS + ROA_D_FS + CFO_ROA_FS + LTD_FS + CR_FS + DILUTION_FS + GM_FS + ATO_FS     
        print(f'{ticker} F_Score is {f_score}')
        F_score_tgt_yr.append(f_score)
        
    F_score["F_score"] = F_score_tgt_yr
        
    return F_score
    